In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models
import en_core_web_sm
from collections import OrderedDict
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = en_core_web_sm.load()

In [2]:
corpus = pd.read_csv('corpus')

In [3]:
corpus

,Unnamed: 0,resource_id,description,full_text,clean_text,tokens
0,0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,connectivity frank kammer hanjo aubig chapter ...,"['connectivity', 'frank', 'kammer', 'hanjo', '..."
1,1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,network science web mandate structural analysi...,"['network', 'science', 'web', 'mandate', 'stru..."
2,2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality index dirk kosch utzki katharina an...,"['centrality', 'index', 'dirk', 'kosch', 'utzk..."
3,3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,social learning network module network science...,"['social', 'learning', 'network', 'module', 'n..."
4,4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,chapter matching market seen number way thinki...,"['chapter', 'matching', 'market', 'seen', 'num..."
...,...,...,...,...,...,...
62,62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,chapter link analysis web search searching web...,"['chapter', 'link', 'analysis', 'web', 'search..."
63,63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,network comparison michael baur marc benkert f...,"['network', 'comparison', 'michael', 'baur', '..."
64,64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,introduction ulrik brandes thomas erlebach man...,"['introduction', 'ulrik', 'brandes', 'thomas',..."
65,65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley value recap cooperating agent agent wi...,"['shapley', 'value', 'recap', 'cooperating', '..."


In [4]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=20):
        """Print top number keywords"""
        keywords = []
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
#             print(key + ' - ' + str(value))
            keywords.append(key)
            if i > number:
                break
        return keywords
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [5]:
obj =  TextRank4Keyword()

In [ ]:
keywords = []
for i in range(len(corpus)): 
    text = corpus['clean_text'][i]
    obj.analyze(text)
    a = obj.get_keywords(30)
    keywords.append(a)

In [ ]:
keywords = pd.DataFrame(keywords)

In [ ]:
df = pd.concat([corpus['resource_id'], keywords], axis=1)